Daniel Ross  
October, 2022  
HyperMaze Class Creation

In [1]:
import pandas as pd
import numpy as np
import random

import igraph as ig

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection, Poly3DCollection

In [12]:
# Instantiate Maze object
maze1 = HyperMaze(3)
# Generate maze path and walls
maze1.buildMaze()
# Plot maze
maze1.plot()

In [14]:
class HyperMaze:
    """Hyper Maze Class"""
    
    def __init__(self, dim):
        self.dim = dim
        self.g = ig.Graph.Lattice([dim,dim,dim], circular=False)
        self.g_vertex_df = pd.DataFrame()
        self.g_edge_df = pd.DataFrame()
        self.wall_df = pd.DataFrame()
      
    def buildMaze(self):
        self.g.es["weight"] = [random.randint(1, 77) for _ in self.g.es]
        
        mst = self.g.spanning_tree(weights=self.g.es["weight"], return_tree=False)
    
        self.g_vertex_df = self.g.get_vertex_dataframe()
        self.g_vertex_df.index.names = ['vertexid']
        self.g_vertex_df['vertex'] = self.g_vertex_df.index
        
        def get_x(v, d): return v % d
    
        def get_y(v, d): return (v // d) % d

        def get_z(v, d): 
            for i in range(d): 
                if v < ((d**2) * (i+1)): 
                    return i
                
        def get_cartesian_coords(v, d):
            x = get_x(v, d)
            y = get_y(v, d)
            z = get_z(v, d)
            return (x, y, z)
                
        self.g_vertex_df['x'] = self.g_vertex_df['vertex'].apply(get_x, d=self.dim)
        self.g_vertex_df['y'] = self.g_vertex_df['vertex'].apply(get_y, d=self.dim)
        self.g_vertex_df['z'] = self.g_vertex_df['vertex'].apply(get_z, d=self.dim)
        
        self.g_edge_df = pd.DataFrame(self.g.get_edge_dataframe())
        self.g_edge_df.index.names = ['edge']
        self.g_edge_df.columns = ['a', 'b', 'w']
        self.g_edge_df["ab"] = list(zip(self.g_edge_df['a'], self.g_edge_df['b']))
        
        self.g_edge_df['mst'] = self.g_edge_df.index.isin(mst)

        mst_edges = self.g_edge_df.loc[self.g_edge_df['mst'], ['a', 'b']]
        mst_edges = list(zip(mst_edges['a'], mst_edges['b']))
        
        sp = ig.Graph([self.dim, self.dim, self.dim], edges=mst_edges)
        shortest_path = sp.get_shortest_paths(0,(self.dim**3)-1, output="epath")
        
        sp_edge_df = sp.get_edge_dataframe()
        sp_edge_df['path'] = sp_edge_df.index.isin(shortest_path[0])
        sp_edge_df = sp_edge_df.loc[sp_edge_df['path'],:]

        sp_edge_df['ab'] = list(zip(sp_edge_df['source'], sp_edge_df['target']))
        sp_verts = list(sp_edge_df['ab'])

        def in_sp(v): return v in sp_verts

        self.g_edge_df['path'] = self.g_edge_df['ab'].apply(in_sp)

        self.g_edge_df['a_xyz'] = self.g_edge_df['a'].apply(get_cartesian_coords, d=self.dim)
        self.g_edge_df['b_xyz'] = self.g_edge_df['b'].apply(get_cartesian_coords, d=self.dim)
        self.g_edge_df['ab_xyz'] = list(zip(self.g_edge_df['a_xyz'], self.g_edge_df['b_xyz']))
        
        def wall_verts(v):
            """kludgy, planes draw as triangles not squares unless careful with ordering of points in list"""
            if v[0][0] != v[1][0]:
                x = [(v[0][0] + v[1][0])/2] * 4
                y = [v[0][1] - .5, v[1][1] + .5]
                y=y+y[::-1]
                z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
            elif v[0][1] != v[1][1]:
                x= [v[0][0] - .5, v[1][0] + .5]
                x=x+x[::-1]
                y = [(v[0][1] + v[1][1])/2]*4
                z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
            else:
                x= [v[0][0] - .5, v[1][0] + .5]
                x=x+x[::-1]
                y = [v[0][1] - .5]*2 + [v[1][1] + .5]*2     
                z = [(v[0][2] + v[1][2])/2]*4
            #coord = np.array(np.meshgrid(x,y,z)).T.reshape(-1,3)
            #coord = list(map(tuple,coord))
            return list(zip(x,y,z))
        
        self.g_edge_df['wall'] = self.g_edge_df['ab_xyz'].apply(wall_verts)

        self.wall_df = self.g_edge_df.loc[self.g_edge_df['mst']==False, 'wall']
        
        
    def plot(self, in_notebook=False,
                   title="default",
                   mst_color="purple", sp_color="forestgreen", 
                   v_color="purple", vgoals_color="forestgreen", 
                   v_size=10, vgoals_size=50,
                   wall_face_color = "grey",
                   wall_face_alpha=0.2,
                   wall_edge_color = "black",
                   maze_figsize=15, pixdensity=400):
        
        if title=="default":
            title = "HyperMaze " + str(self.dim) + "x" + str(self.dim)
            
        self.g_edge_df['color'] = "white"
        self.g_edge_df.loc[self.g_edge_df['mst'], 'color'] = mst_color
        self.g_edge_df.loc[self.g_edge_df['path'], 'color'] = sp_color

        self.g_vertex_df['vcolor'] = v_color
        self.g_vertex_df.loc[[0,len(self.g_vertex_df)-1], 'vcolor'] = vgoals_color
        self.g_vertex_df['vsize'] = v_size
        self.g_vertex_df.loc[[0,len(self.g_vertex_df)-1], 'vsize'] = vgoals_size

        wall_color = wall_face_color
        wall_edge_color = wall_edge_color
        
        if in_notebook:
            %matplotlib notebook
        else:
            %matplotlib qt
        
        fig = plt.figure(title, figsize=(maze_figsize,maze_figsize), dpi=pixdensity)
        ax = fig.add_subplot(projection='3d')

        ax.set_axis_off()
        plt.tight_layout()

        ax.scatter(self.g_vertex_df['x'], self.g_vertex_df['y'], self.g_vertex_df['z'], 
                   c=self.g_vertex_df['vcolor'], 
                   s=self.g_vertex_df['vsize'])

        ax.add_collection3d(Line3DCollection(np.array(self.g_edge_df.loc[self.g_edge_df['mst'],'ab_xyz']),
                                             colors=self.g_edge_df.loc[self.g_edge_df['mst'],'color']))

        walls = ax.add_collection3d(Poly3DCollection(list(self.wall_df), linewidths=1, alpha=wall_face_alpha))
        walls.set_facecolor(wall_color)
        walls.set_edgecolor(wall_edge_color)
        ax.add_collection3d(walls)

        ax.set_xlim([-(0.5),(self.dim)])
        ax.set_ylim([-(0.5),(self.dim)])
        ax.set_zlim([-(0.5),(self.dim)])